In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from utils.dataset import get_dataloaders
from utils.metrics import evaluate_model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = 'Pets'
train_loader, val_loader, class_names = get_dataloaders(data_dir)
num_classes = len(class_names)

In [ ]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")

In [ ]:
report, cm = evaluate_model(model, val_loader, device, class_names)
print("\nTransfer Learning Report:\n")
print(report)

In [ ]:
torch.save(model.state_dict(), 'transfer_best_model.pth')
print("Saved transfer_best_model.pth")